# PDF-to-Embedding Pipeline
## Procesare 15GB Manuale Școlare → Supabase pgvector

Notebook principal pentru Kaggle P100 GPU processing

**Setup:**
1. Accelerator: GPU (P100)
2. Internet: Enable
3. Secrets: SUPABASE_URL, SUPABASE_ANON_KEY
4. Attached dataset: materiale-didactice folder

**Estimated runtime:** 18-24 ore pentru 15GB

## Cell 1: Setup & Dependencies

In [ ]:
# Setup initial - verify GPU și install dependencies

import os
import sys
import logging
from pathlib import Path

# Verify GPU
print("Checking GPU availability...")
!nvidia-smi

print("\nInstalling dependencies...")
!pip install -q PyMuPDF>=1.23.0 paddleocr>=2.7.0 sentence-transformers>=2.2.2 supabase>=2.0.0 tqdm pyyaml

print("\n✅ Dependencies installed successfully")

## Cell 2: Import Modules & Configuration

In [ ]:
# Import all modules
from kaggle_secrets import UserSecretsClient
import yaml
import logging
from tqdm import tqdm
from pathlib import Path
import json
import time
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Obține Supabase secrets
logger.info("Loading Supabase credentials from secrets...")
secret = UserSecretsClient()

try:
    SUPABASE_URL = secret.get_secret('SUPABASE_URL')
    SUPABASE_KEY = secret.get_secret('SUPABASE_ANON_KEY')
    logger.info(f"✅ Loaded credentials for: {SUPABASE_URL[:30]}...")
except Exception as e:
    logger.error(f"❌ Failed to load secrets: {e}")
    logger.error("Make sure to add SUPABASE_URL and SUPABASE_ANON_KEY in notebook Settings → Secrets")
    raise

# Load config
logger.info("Loading configuration...")
with open('/kaggle/input/') as f:  # Will be attached dataset
    try:
        # Try to load from config if available
        config_path = '/kaggle/working/config.yaml'  # Local copy
    except:
        # Use defaults
        pass

config = {
    'pdf': {'chunk_size': 500, 'overlap': 50, 'min_chunk_length': 50},
    'ocr': {'enabled': True, 'languages': ['ro', 'en'], 'min_image_size': 51200, 'priority_threshold': 0.8},
    'embeddings': {'model': 'paraphrase-multilingual-mpnet-base-v2', 'batch_size': 128, 'device': 'cuda'},
    'supabase': {'batch_size': 10000, 'max_retries': 3}
}

logger.info(f"✅ Configuration loaded")
logger.info(f"  Chunk size: {config['pdf']['chunk_size']} chars")
logger.info(f"  Embedding model: {config['embeddings']['model']}")
logger.info(f"  Device: {config['embeddings']['device']}")

In [ ]:
# Import all modules
from kaggle_secrets import UserSecretsClient
import yaml
import logging
from tqdm import tqdm
from pathlib import Path
import json
import time
from datetime import datetime

# Setup logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Obține Supabase secrets
logger.info("Loading Supabase credentials from secrets...")
secret = UserSecretsClient()

try:
    SUPABASE_URL = secret.get_secret('SUPABASE_URL')
    SUPABASE_KEY = secret.get_secret('SUPABASE_ANON_KEY')
    logger.info(f"✅ Loaded credentials for: {SUPABASE_URL[:30]}...")
except Exception as e:
    logger.error(f"❌ Failed to load secrets: {e}")
    logger.error("Make sure to add SUPABASE_URL and SUPABASE_ANON_KEY in notebook Settings → Secrets")
    raise

# Load config
logger.info("Loading configuration...")
with open('/kaggle/input/') as f:  # Will be attached dataset
    try:
        # Try to load from config if available
        config_path = '/kaggle/working/config.yaml'  # Local copy
    except:
        # Use defaults
        pass

config = {
    'pdf': {'chunk_size': 500, 'overlap': 50, 'min_chunk_length': 50},
    'ocr': {'enabled': True, 'languages': ['ro', 'en'], 'min_image_size': 51200, 'priority_threshold': 0.8},
    'embeddings': {'model': 'paraphrase-multilingual-mpnet-base-v2', 'batch_size': 128, 'device': 'cuda'},
    'supabase': {'batch_size': 10000, 'max_retries': 3}
}

logger.info(f"✅ Configuration loaded")
logger.info(f"  Chunk size: {config['pdf']['chunk_size']} chars")
logger.info(f"  Embedding model: {config['embeddings']['model']}")
logger.info(f"  Device: {config['embeddings']['device']}")

## Cell 3: Find & List PDFs

In [ ]:
# Find all PDFs in mounted dataset
logger.info("\nSearching for PDFs...")

# Locate dataset folder
kaggle_input = Path('/kaggle/input')
dataset_folders = [d for d in kaggle_input.iterdir() if d.is_dir()]

if not dataset_folders:
    logger.error("❌ No dataset folder found in /kaggle/input/")
    logger.error("Make sure to attach dataset before running!")
    raise FileNotFoundError("Dataset not found")

# Use first dataset folder
pdf_folder = dataset_folders[0]
logger.info(f"Using dataset: {pdf_folder.name}")

# Find all PDFs recursively
all_pdfs = list(pdf_folder.glob('**/*.pdf'))

logger.info(f"\n✅ Found {len(all_pdfs)} PDFs")
logger.info(f"Sample PDFs:")
for pdf in all_pdfs[:5]:
    size_mb = pdf.stat().st_size / 1024 / 1024
    logger.info(f"  - {pdf.relative_to(pdf_folder)} ({size_mb:.1f} MB)")

if len(all_pdfs) > 5:
    logger.info(f"  ... and {len(all_pdfs) - 5} more")

# Calculate total size
total_size_gb = sum(p.stat().st_size for p in all_pdfs) / 1024 / 1024 / 1024
logger.info(f"\nTotal dataset size: {total_size_gb:.1f} GB")
logger.info(f"Processing time estimate: 18-25 hours")

## Cell 4: Define Processing Functions

In [ ]:
# Import processing modules directly into notebook
# (Alternative: download from GitHub or copy from scripts folder)

import fitz  # PyMuPDF
from sentence_transformers import SentenceTransformer
from supabase import create_client
import numpy as np
import hashlib
import re

logger.info("Processing functions initialized")

# ===== PDF EXTRACTION =====
def extract_text_simple(pdf_path: str) -> dict:
    """Simplu PDF text extraction cu PyMuPDF"""
    try:
        pdf = fitz.open(pdf_path)
        text = ""
        for page in pdf:
            text += page.get_text()
        pdf.close()
        return {'text': text, 'pages': len(pdf), 'status': 'success'}
    except Exception as e:
        return {'text': '', 'pages': 0, 'status': 'error', 'error': str(e)}

# ===== TEXT CHUNKING =====
def chunk_text(text: str, chunk_size: int = 500, overlap: int = 50) -> list:
    """Splittează text în chunks cu overlap"""
    sentences = re.split(r'(?<=[.!?])\s+', text)
    chunks = []
    current = ""
    
    for sent in sentences:
        if len(current) + len(sent) > chunk_size:
            if current:
                chunks.append(current)
            current = sent
        else:
            current += " " + sent if current else sent
    
    if current:
        chunks.append(current)
    
    return chunks

# ===== DEDUPLICATION =====
def deduplicate_chunks(chunks: list) -> list:
    """Elimina duplicate chunks (headers/footers)"""
    seen_hashes = set()
    unique = []
    
    for chunk in chunks:
        chunk_hash = hashlib.md5(chunk.encode()).hexdigest()
        if chunk_hash not in seen_hashes and len(chunk) >= 50:
            seen_hashes.add(chunk_hash)
            unique.append(chunk)
    
    return unique

# ===== EMBEDDING GENERATION =====
def init_embedding_model(device='cuda'):
    """Initialize sentence-transformers model"""
    logger.info(f"Loading embedding model on {device}...")
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2', device=device)
    return model

def generate_embeddings(texts: list, model, batch_size: int = 128) -> np.ndarray:
    """Generate embeddings for texts"""
    embeddings = model.encode(texts, batch_size=batch_size, convert_to_numpy=True)
    return embeddings

# ===== SUPABASE UPLOAD =====
def init_supabase(url: str, key: str):
    """Initialize Supabase client"""
    return create_client(url, key)

def upload_vectors_batch(supabase, vectors: list, batch_size: int = 10000) -> dict:
    """Upload vectors to Supabase in batches"""
    success = 0
    failed = 0
    
    for i in range(0, len(vectors), batch_size):
        batch = vectors[i:i+batch_size]
        try:
            response = supabase.table('document_embeddings').insert(batch).execute()
            success += len(batch)
        except Exception as e:
            logger.warning(f"Batch {i//batch_size} failed: {e}")
            failed += len(batch)
    
    return {'success': success, 'failed': failed}

logger.info("✅ All processing functions ready")

## Cell 5: Main Processing Loop

In [ ]:
# Main processing pipeline
logger.info("\n" + "="*70)
logger.info("STARTING MAIN PROCESSING PIPELINE")
logger.info("="*70)

# Initialize components
logger.info("\nInitializing components...")
embedding_model = init_embedding_model(device=config['embeddings']['device'])
supabase = init_supabase(SUPABASE_URL, SUPABASE_KEY)
logger.info("✅ Components initialized")

# Statistics
stats = {
    'processed_pdfs': 0,
    'failed_pdfs': 0,
    'total_text_chars': 0,
    'total_chunks': 0,
    'total_embeddings': 0,
    'uploaded_vectors': 0,
    'failed_uploads': 0,
    'start_time': time.time()
}

# Process each PDF
logger.info(f"\nProcessing {len(all_pdfs)} PDFs...")

# Optional: limit for testing
TEST_MODE = False  # Set to True to process only 10 PDFs
pdfs_to_process = all_pdfs[:10] if TEST_MODE else all_pdfs

with tqdm(total=len(pdfs_to_process), desc="PDF Processing") as pbar:
    current_batch = []
    batch_size = config['supabase']['batch_size']
    
    for pdf_path in pdfs_to_process:
        try:
            # 1. Extract text
            result = extract_text_simple(str(pdf_path))
            
            if result['status'] != 'success':
                stats['failed_pdfs'] += 1
                pbar.update(1)
                continue
            
            text = result['text']
            stats['total_text_chars'] += len(text)
            stats['processed_pdfs'] += 1
            
            # 2. Chunk text
            chunks = chunk_text(text, config['pdf']['chunk_size'], config['pdf']['overlap'])
            chunks = deduplicate_chunks(chunks)
            stats['total_chunks'] += len(chunks)
            
            # 3. Generate embeddings
            embeddings = generate_embeddings(chunks, embedding_model, config['embeddings']['batch_size'])
            stats['total_embeddings'] += len(embeddings)
            
            # 4. Prepare for upload
            pdf_name = pdf_path.relative_to(pdf_folder)
            
            for j, (chunk, emb) in enumerate(zip(chunks, embeddings)):
                vector_item = {
                    'chunk_id': f"{pdf_path.stem}_{j}",
                    'text': chunk[:5000],
                    'embedding': emb.tolist(),
                    'source_pdf': str(pdf_name),
                    'page_num': 1,
                    'clasa': 1,
                    'materie': 'Various',
                    'capitol': 'Chapter',
                    'chunk_hash': hashlib.md5(chunk.encode()).hexdigest(),
                    'has_images': False
                }
                current_batch.append(vector_item)
            
            # 5. Upload batch if full
            if len(current_batch) >= batch_size:
                result = upload_vectors_batch(supabase, current_batch, batch_size)
                stats['uploaded_vectors'] += result['success']
                stats['failed_uploads'] += result['failed']
                current_batch = []
            
            pbar.update(1)
            
        except Exception as e:
            logger.warning(f"Error processing {pdf_path.name}: {e}")
            stats['failed_pdfs'] += 1
            pbar.update(1)
    
    # Upload remaining batch
    if current_batch:
        result = upload_vectors_batch(supabase, current_batch, batch_size)
        stats['uploaded_vectors'] += result['success']
        stats['failed_uploads'] += result['failed']

logger.info("✅ Main processing complete")

## Cell 6: Print Statistics

In [ ]:
# Print final statistics
elapsed = time.time() - stats['start_time']

logger.info("\n" + "="*70)
logger.info("PROCESSING COMPLETE - FINAL STATISTICS")
logger.info("="*70)

logger.info(f"\nProcessing time: {elapsed/3600:.1f} hours")
logger.info(f"\nPDF Processing:")
logger.info(f"  ✓ Successful: {stats['processed_pdfs']}")
logger.info(f"  ✗ Failed: {stats['failed_pdfs']}")
logger.info(f"\nText Statistics:")
logger.info(f"  Total text: {stats['total_text_chars']:,} characters")
logger.info(f"  Average per PDF: {stats['total_text_chars']//max(stats['processed_pdfs'],1):,} chars")
logger.info(f"\nChunking:")
logger.info(f"  Total chunks: {stats['total_chunks']:,}")
logger.info(f"  Chunks per PDF: {stats['total_chunks']//max(stats['processed_pdfs'],1)}")
logger.info(f"\nEmbeddings:")
logger.info(f"  Total vectors: {stats['total_embeddings']:,}")
logger.info(f"  Vector dimension: 768")
logger.info(f"\nSupabase Upload:")
logger.info(f"  ✓ Uploaded: {stats['uploaded_vectors']:,}")
logger.info(f"  ✗ Failed: {stats['failed_uploads']}")

logger.info(f"\n" + "="*70)
logger.info(f"Estimated database size: {(stats['total_embeddings'] * 3.5 / 1024):.0f} MB")
logger.info(f"Vectors ready for semantic search!")
logger.info(f"="*70)

## Cell 7: Verify Supabase Upload

In [ ]:
# Verify vectors in Supabase
logger.info("\nVerifying Supabase...")

try:
    response = supabase.table('document_embeddings').select("count", count="exact").execute()
    db_vectors = response.count
    
    logger.info(f"✅ Total vectors in Supabase: {db_vectors:,}")
    
    if db_vectors == stats['uploaded_vectors']:
        logger.info("✅ Upload verification PASSED")
    else:
        logger.warning(f"⚠️  Expected {stats['uploaded_vectors']}, but found {db_vectors}")
        
except Exception as e:
    logger.error(f"❌ Verification failed: {e}")

## Cell 8: (Optional) Create HNSW Index

In [ ]:
# Create HNSW index for similarity search (if not already created)
logger.info("\nNote: HNSW index creation happens in Supabase SQL Editor")
logger.info("Run this SQL to create index (one-time):")
logger.info("""
CREATE INDEX IF NOT EXISTS idx_embedding_hnsw
ON document_embeddings
USING hnsw (embedding vector_cosine_ops)
WITH (m = 16, ef_construction = 64);
""")
logger.info("Index creation takes 30-60 min for 600k vectors (one-time cost)")

## Cell 9: (Optional) Test Similarity Search

In [ ]:
# Test similarity search (if vectors exist)
if stats['uploaded_vectors'] > 0:
    logger.info("\nTesting similarity search...")
    
    # Generate query embedding
    query_text = "Cum se calculează aria unui pătrat?"  # Example Romanian query
    query_emb = embedding_model.encode([query_text])[0].tolist()
    
    try:
        # Use RPC function (defined in SQL)
        results = supabase.rpc('match_documents', {
            'query_embedding': query_emb,
            'match_count': 5
        }).execute()
        
        logger.info(f"\nQuery: '{query_text}'")
        logger.info(f"Results:")
        
        if results.data:
            for i, result in enumerate(results.data[:3]):
                text = result.get('text', '')[:60]
                similarity = result.get('similarity', 0)
                logger.info(f"  {i+1}. {text}... (similarity: {similarity:.1%})")
        else:
            logger.info("  No results found (index might not be created yet)")
            
    except Exception as e:
        logger.warning(f"Query failed: {e}")
        logger.info("(This is OK if RPC function not created yet)")
else:
    logger.info("No vectors uploaded, skipping search test")

## DONE! 🎉

In [ ]:
logger.info("\n" + "="*70)
logger.info("NEXT STEPS")
logger.info("="*70)
logger.info("""
1. ✅ Embeddings are in Supabase
2. ✅ Run SQL to create HNSW index (Supabase SQL Editor)
3. ✅ Wait 30-60 min for index to build
4. ✅ Now ready to use in AI tutoring system!

Your vectors are permanent in Supabase free tier.
Storage used: ~300-500 MB / 500 MB available

For your AI app:
- Generate query embedding (same model)
- Call Supabase RPC: match_documents()
- Get top-10 similar chunks
- Feed to LLM for context
""")
logger.info("="*70)